In [33]:
import pandas as pd

In [32]:
df = pd.read_excel("data/my_data.xlsx")

In [34]:
from arml import unit_root_test, plot_PACF_ACF

In [35]:
unit_root_test(my_df["electricity"], method="ADF")

ADF p-value: 0.000000 and data is stationary at 5% significance level


(1.5469596472188275e-29, None)

In [36]:
plot_PACF_ACF(my_df["electricity"], lag_num=19)

NameError: name 'plt' is not defined